# Imports

In [1]:
import numpy as np

In [2]:
%load_ext lab_black

# Definitions

In [12]:
m = 50
n = 5
k = 3
lr = 1.0
batch = 50

In [13]:
np.random.seed(0)
X_matrix = np.random.randn(m, n).astype(np.float32)
y = np.random.randint(k, size=(m,)).astype(np.uint8)
theta_matrix = np.zeros((n, k), dtype=np.float32)

In [14]:
theta_matrix.flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [15]:
y

array([1, 0, 2, 0, 0, 1, 2, 1, 1, 1, 0, 0, 2, 0, 0, 2, 1, 2, 2, 0, 2, 2,
       0, 2, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 0, 1, 2, 0, 1, 2,
       1, 2, 1, 2, 1, 2], dtype=uint8)

In [16]:
X_matrix.flatten()

array([ 1.7640524 ,  0.4001572 ,  0.978738  ,  2.2408931 ,  1.867558  ,
       -0.9772779 ,  0.95008844, -0.1513572 , -0.10321885,  0.41059852,
        0.14404356,  1.4542735 ,  0.7610377 ,  0.12167501,  0.44386324,
        0.33367434,  1.4940791 , -0.20515826,  0.3130677 , -0.85409576,
       -2.5529897 ,  0.6536186 ,  0.8644362 , -0.742165  ,  2.2697546 ,
       -1.4543657 ,  0.04575852, -0.18718386,  1.5327792 ,  1.4693588 ,
        0.15494743,  0.37816253, -0.88778573, -1.9807965 , -0.34791216,
        0.15634897,  1.2302907 ,  1.2023798 , -0.3873268 , -0.30230275,
       -1.048553  , -1.420018  , -1.7062702 ,  1.9507754 , -0.5096522 ,
       -0.4380743 , -1.2527953 ,  0.7774904 , -1.6138978 , -0.21274029,
       -0.89546657,  0.3869025 , -0.51080513, -1.1806322 , -0.02818223,
        0.42833188,  0.06651722,  0.3024719 , -0.6343221 , -0.36274117,
       -0.67246044, -0.35955316, -0.8131463 , -1.7262826 ,  0.17742614,
       -0.40178093, -1.6301984 ,  0.46278226, -0.9072984 ,  0.05

# C++

In [108]:
theta = theta_matrix.reshape(n * k)
X = X.reshape(m * n)
theta.shape, X.shape, y.shape

((15,), (250,), (50,))

In [109]:
X_transpose = np.empty(n * m)
H = np.empty(m * k)
Z_tmp = np.empty(m * k)
Z = np.empty(m * k)
I = np.empty(m * k)
sum_exp = np.empty_like(y, dtype="float")
grad = np.random.randn(n * k)
X_transpose.shape, H.shape, Z_tmp.shape, Z.shape, I.shape, sum_exp.shape, grad.shape

((250,), (150,), (150,), (150,), (150,), (50,), (15,))

In [110]:
for i in range(n):
    for j in range(m):
        X_transpose[i * m + j] = X[j * n + i]

In [111]:
for iteration in range(0, m, batch):
    start_index = iteration * batch
    for i in range(m):
        sum_exp[i] = 0.0
        for j in range(k):
            H[i * k + j] = 0.0
            for l in range(n):
                H[i * k + j] += X[(start_index + i) * n + l] * theta[l * k + j]
            Z_tmp[i * k + j] = np.exp(H[i * k + j])
            # print(f"Exp = {Z_tmp[i * k + j]}")
            sum_exp[i] += Z_tmp[i * k + j]

    for i in range(m):
        for j in range(k):
            Z[i * k + j] = Z_tmp[i * k + j] / sum_exp[i]
            # print(f"Prob = {Z[i * k + j]}")
            I[i * k + j] = 0.0
        I[i * k + y[i]] = 1.0

    for i in range(n):
        for j in range(k):
            grad[i * k + j] = 0.0
            for l in range(batch):
                grad[i * k + j] += X_transpose[i * m + (start_index + l)] * (
                    Z[(l + start_index) * k + j] - I[(l + start_index) * k + j]
                )
            print(f"Grad ({i, j}) = {grad[i * k + j]}")
            theta[i * k + j] -= (lr * grad[i * k + j]) / batch

Grad ((0, 0)) = 1.2629730924963969
Grad ((0, 1)) = 5.732612252235414
Grad ((0, 2)) = -6.995585344731808
Grad ((1, 0)) = -1.145653979231914
Grad ((1, 1)) = 4.238694470065336
Grad ((1, 2)) = -3.093040490833422
Grad ((2, 0)) = -2.1933788868288207
Grad ((2, 1)) = -0.3714889871577427
Grad ((2, 2)) = 2.564867873986562
Grad ((3, 0)) = 3.8056764950354913
Grad ((3, 1)) = -2.4706098834673567
Grad ((3, 2)) = -1.3350666115681333
Grad ((4, 0)) = -0.8459034735957784
Grad ((4, 1)) = -0.6703241846213737
Grad ((4, 2)) = 1.516227658217152


In [112]:
theta

array([-0.02525946, -0.11465225,  0.13991171,  0.02291308, -0.08477389,
        0.06186081,  0.04386758,  0.00742978, -0.05129736, -0.07611353,
        0.0494122 ,  0.02670133,  0.01691807,  0.01340648, -0.03032455],
      dtype=float32)

In [21]:
theta_cached = np.copy(theta)

# Python Version

In [23]:
# Forward
X_batch = X_matrix
y_batch = y
H_python = X_batch.dot(theta_matrix)
Z_python = np.exp(H_python) / np.sum(np.exp(H_python), axis=1, keepdims=True)

# Backward
I_python = np.zeros((m, k))
I_python[np.arange(m), y_batch] = 1
grad_python = X_batch.transpose().dot(Z_python - I_python)
theta_matrix -= (lr * grad_python) / m

In [17]:
for i in range(0, m, batch):
    # Forward
    X_batch = X_matrix[i : i + batch]
    y_batch = y[i : i + batch]
    H_python = X_batch.dot(theta_matrix)
    Z_python = np.exp(H_python) / np.sum(np.exp(H_python), axis=1, keepdims=True)

    # Backward
    I_python = np.zeros((batch, k))
    I_python[np.arange(batch), y_batch] = 1
    grad_python = X_batch.transpose().dot(Z_python - I_python)
    theta_matrix -= (lr * grad_python) / batch

In [18]:
theta_matrix

array([[-0.02525946, -0.11465225,  0.13991171],
       [ 0.02291308, -0.08477389,  0.06186081],
       [ 0.04386758,  0.00742978, -0.05129736],
       [-0.07611353,  0.0494122 ,  0.02670133],
       [ 0.01691807,  0.01340648, -0.03032455]], dtype=float32)

In [19]:
H_python

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0

In [20]:
Z_python

array([[0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.

In [23]:
I_python

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0

In [24]:
Z_python - I_python

array([[ 0.33333334, -0.66666666,  0.33333334],
       [-0.66666666,  0.33333334,  0.33333334],
       [ 0.33333334,  0.33333334, -0.66666666],
       [-0.66666666,  0.33333334,  0.33333334],
       [-0.66666666,  0.33333334,  0.33333334],
       [ 0.33333334, -0.66666666,  0.33333334],
       [ 0.33333334,  0.33333334, -0.66666666],
       [ 0.33333334, -0.66666666,  0.33333334],
       [ 0.33333334, -0.66666666,  0.33333334],
       [ 0.33333334, -0.66666666,  0.33333334],
       [-0.66666666,  0.33333334,  0.33333334],
       [-0.66666666,  0.33333334,  0.33333334],
       [ 0.33333334,  0.33333334, -0.66666666],
       [-0.66666666,  0.33333334,  0.33333334],
       [-0.66666666,  0.33333334,  0.33333334],
       [ 0.33333334,  0.33333334, -0.66666666],
       [ 0.33333334, -0.66666666,  0.33333334],
       [ 0.33333334,  0.33333334, -0.66666666],
       [ 0.33333334,  0.33333334, -0.66666666],
       [-0.66666666,  0.33333334,  0.33333334],
       [ 0.33333334,  0.33333334, -0.666

In [22]:
grad_pythonn

array([[ 1.26297303,  5.73261219, -6.9955854 ],
       [-1.1456539 ,  4.23869455, -3.09304042],
       [-2.19337888, -0.37148898,  2.56486788],
       [ 3.80567649, -2.47060989, -1.33506662],
       [-0.84590342, -0.67032413,  1.51622772]])

In [27]:
np.allclose(
    np.array(
        [
            1.262973,
            5.732611,
            -6.995585,
            -1.145655,
            4.238695,
            -3.093040,
            -2.193379,
            -0.371489,
            2.564868,
            3.805676,
            -2.470610,
            -1.335067,
            -0.845903,
            -0.670324,
            1.516228,
        ]
    ),
    grad_python.flatten(),
)

True

In [31]:
theta_matrix

array([[-0.02525946, -0.11465225,  0.13991171],
       [ 0.02291308, -0.08477389,  0.06186081],
       [ 0.04386758,  0.00742978, -0.05129736],
       [-0.07611353,  0.0494122 ,  0.02670133],
       [ 0.01691807,  0.01340648, -0.03032455]], dtype=float32)

# Testing Matrix Multiplication

- [x] H
- [x] Z_tmp
- [x] sum_exp
- [x] Z
- [x] I
- [x] X_transpose

In [74]:
np.allclose(X_matrix.dot(theta_matrix), H.reshape((m, k)))

True

In [156]:
np.allclose(H_python, H.reshape((m, k)))

True

In [159]:
np.allclose(I_python, I.reshape((m, k)))

True

In [162]:
np.allclose(X_matrix.transpose(), X_transpose.reshape((n, m)))

True

In [163]:
np.allclose(grad_python, grad.reshape((n, k)))

True

In [164]:
np.allclose(theta_python, theta.reshape((n, k)))

True